<a href="https://colab.research.google.com/github/madhugraj/Droupout/blob/master/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import keras
import tensorflow as tf

#config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
#sess = tf.compat.v1.Session(config=config) 
#tf.compat.v1.keras.backend.set_session(sess)

from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation,concatenate,GlobalAveragePooling2D,Flatten,MaxPooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np
from keras.datasets import cifar10
from tqdm import tqdm
from keras.utils import plot_model
import os
#os.environ["PATH"] += os.pathsep + r'C:\Program Files (x86)\graphviz-2.38\bin/'

import cv2 as cv2
from imutils import paths
import random
from tensorflow.keras.preprocessing.image import img_to_array

import glob,progressbar,tqdm
from keras.preprocessing.image import load_img,array_to_img,img_to_array

import matplotlib.pyplot as plt

In [ ]:
!unzip /content/Small_Data.zip

In [ ]:
!unzip /content/test.zip

In [ ]:
x_train2= sorted(glob.glob(r'/content/Small_Data/*.*'))
valid_data= sorted(glob.glob(r'/content/test/*.*'))

x_tr =[]
valid =[]

def img_preprocess(img):
    img=load_img(img,color_mode='grayscale',target_size=(512,512))
    img= img_to_array(img).astype('float32')/255
    return img

In [ ]:
from tqdm import tqdm

for img in tqdm(x_train2):
    x_tr.append(img_preprocess(img))
    
x_tr= np.array(x_tr)
 


for img in tqdm(valid_data):
    valid.append(img_preprocess(img))
    
valid= np.array(valid)

print('Train Shape:',x_tr.shape)
print('Test Shape:',valid.shape)

In [ ]:
layer_in = Input(shape=(512,512,1),name= 'InputLayer')

# 1x1 convolution
conv1 = Conv2D(128, (1,1),padding='same',activation='relu',name='Conv_1x1')(layer_in)

#3x3 conv
conv3 =  Conv2D(128, (1,1),padding='same',activation='relu')(layer_in)
conv3 =  Conv2D(128, (3,3),padding='same',activation='relu',name='Conv_3x3')(conv3)

#5x5 conv
conv5 =  Conv2D(128,(1,1),padding='same',activation='relu')(layer_in)
conv5 =  Conv2D(128,(5,5),padding='same',activation='relu',name='Conv_5x5')(conv5)

#3x3 Max pooling

pool = MaxPooling2D((3,3),strides=(1,1),padding='same',name='max_pool_3x3')(layer_in)
pool =  Conv2D(128,(1,1),padding='same',activation='relu')(pool)
output= concatenate([conv1,conv3,conv5,pool],axis=-1)

encode = MaxPooling2D((2,2),padding='same',name='pool1')(output)
encode = Conv2D(256,(2,2),activation='relu',padding='same',name='concat_256')(encode)

## Latent View
latent_view    = MaxPooling2D((2, 2), padding='same',name='latent_view')(encode)

decode = UpSampling2D((2,2), name='Upsampling2')(encode)
decode = Conv2D(128,(2,2),activation='relu',padding='same', name='conv3')(decode)
decode = Conv2D(1,(2,2),activation='sigmoid',padding='same',name='conv4')(decode)

autoencoder= Model(inputs= layer_in,outputs=decode)
autoencoder.compile(optimizer='Adam',loss='mse',metrics=['mse','accuracy'])

autoencoder.summary()
plot_model(autoencoder, to_file='model_new_1x1.jpeg', show_layer_names=True, show_shapes=True)

In [ ]:
weights_path= r'/content/Weights/'


'''
es_cb = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
chkpt = weights_path + "Autoencoder_adam_y_512_512_epoch_{}.h5"
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = autoencoder.fit(x_tr, x_tr,
                            batch_size=8,
                            epochs=10,
                            verbose=1,
                            callbacks=[es_cb, cp_cb],
                            shuffle=True)

sample_test= load_img(valid,color_mode="grayscale",target_size=(512,512))
sample_test =img_to_array(sample_test)
sample_test = sample_test.astype('float32')/255.

sample_test = np.expand_dims(sample_test,axis=0)
prection = autoencoder.predict([sample_test,sample_test])

f,ax = plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(np.seqeeze(sample_test),cmap='gray')
ax[1].imshow(prection.reshape(512,512),cmap='gray')
plt.show()
'''

In [ ]:
for i in range(10):
    autoencoder.fit(x_tr,x_tr,epochs=5,batch_size=8)
    autoencoder.save(weights_path + "Autoencoder_adam_y_512_512_epoch_{}.h5",format(i*10))
    ### prediction
    
    sample_test= load_img(valid,color_mode="grayscale",target_size=(512,512))
    sample_test =img_to_array(sample_test)
    sample_test = sample_test.astype('float32')/255.
    
    sample_test = np.expand_dims(sample_test,axis=0)
    prection = autoencoder.predict([sample_test,sample_test])
    
    f,ax = plt.subplots(1,2,figsize=(20,10))
    ax[0].imshow(np.seqeeze(sample_test),cmap='gray')
    ax[1].imshow(prection.reshape(512,512),cmap='gray')
    plt.show()